In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
!pip install --upgrade transformers accelerate

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Trainer,
    default_data_collator
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch
from google.colab import files

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CS224N

## Baseline, Without Fine-Tuning

In [ ]:
# Load NeuralNovel model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm

tokenizer = AutoTokenizer.from_pretrained("NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story")
model = AutoModelForCausalLM.from_pretrained("NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story")

Continuity Error

In [ ]:
model = model.to('cuda')
test_df = pd.read_csv('test_dataset.csv')
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['modified']

    prompt = f"There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of '<nl>' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story}"

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs, max_new_tokens=50)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    responses.append(response)
    print(f"Response: {response}")

df_response = pd.DataFrame(responses, columns=['response'])
df_response.to_csv("test_neuralnovel_responses.csv", index=False)

from google.colab import files
files.download("test_neuralnovel_responses.csv")

Unresolved Error

In [ ]:
model = model.to('cuda')
test_df = pd.read_csv('test_dataset_unresolved.csv').reset_index(drop=True).dropna()

def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line.strip() for line in lines if line.strip()]
  return '\n'.join(non_empty_lines)

test_df['story'] = test_df['story'].apply(remove_empty_lines)
test_df['story_truncated'] = test_df['story_truncated'].apply(remove_empty_lines)

import re
from tqdm import tqdm
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['story_truncated']
    proportion = row['chunk_prop']
    prompt = f"[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: {story} [/INST]"

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs,  max_length=1024)
    print(output)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(response)
    responses.append(response)

df_response = pd.DataFrame(responses, columns=['response'])
df_response.to_csv("test_responses.csv", index=False)

from google.colab import files
files.download("test_responses.csv")

## Fine-Tuning

Dataset Preprocessing

In [ ]:
# Unresolved Error Dataset
train_data = pd.read_csv('train_dataset_unresolved.csv')
question = "Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: "
dataset = Dataset.from_pandas(train_data)

def transform_story(example):
    story_text = example['story']
    error_position =  1 - example['chunk_prop']

    formatted_text = f'[INST] Evaluate the completeness of the given story. Answer only with a proportion, in the form of a decimal. Do not include any other details. Story: {story_text} [/INST] {error_position} '
    return {'text': formatted_text}

transformed_dataset = dataset.map(transform_story)
for i in range(5):
    print(transformed_dataset[i]['text'])

In [ ]:
# Continuity Error Dataset
df = pd.read_csv('train_dataset.csv')
dataset = Dataset.from_pandas(df)

def transform_story(example):
    story_text = example['modified'].rstrip()
    error_position = example['position']

    formatted_text = f'<s>[INST] There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of \'<nl>\' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story_text} [/INST] {error_position} </s>'
    return {'text': formatted_text}

transformed_dataset = dataset.map(transform_story)
for i in range(5):
    print(transformed_dataset[i]['text'])


Fine-tuning Model Parameters

In [ ]:
model_name = "NeuralNovel/Mistral-7B-Instruct-v0.2-Neural-Story"

### QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

### TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

### SFT parameters

# Maximum sequence length to use
max_seq_length = 1024

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

Fine-tuning with QLoRA

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load LoRA configuration specific to Mistral
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=transformed_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("neuralnovel-mistral-7b")

model.eval()

## Evaluation on Test Set

Unresolved Error

In [ ]:
test_df = pd.read_csv('test_dataset_unresolved.csv').reset_index(drop=True).dropna()
def remove_empty_lines(text):
  lines = text.split('\n')
  non_empty_lines = [line.strip() for line in lines if line.strip()]
  return '\n'.join(non_empty_lines)

# Apply the function to the text column
test_df['story'] = test_df['story'].apply(remove_empty_lines)
test_df['story_truncated'] = test_df['story_truncated'].apply(remove_empty_lines)

import re
from tqdm import tqdm
responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['story_truncated']
    proportion = 1 - row['chunk_prop']
    prompt = f"[INST] Estimate how complete the following story is, in terms of plot development. Answer only as a proportion. Do not include any other detaisl. Story: {story} [/INST]"

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs,  max_length=1024)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    print(response)
    responses.append(response)

df_response = pd.DataFrame(responses, columns=['response'])
df_response.to_csv("test_neuralnovel_responses.csv", index=False)

from google.colab import files
files.download("test_neuralnovel_responses.csv")

Continuity Error

In [ ]:
from tqdm.auto import tqdm
test_df = pd.read_csv('test_dataset.csv')

responses = []

for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    story = row['modified']
    prompt = f"There is one major continuity error, a lapse in the self-consistency of the narrative, in the story provided. Count the number of '<nl>' in the story that occurs before the continuity error. Return that number only. Do not return any other information in your response. Story: {story}"

    model_inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    output = model.generate(**model_inputs, max_new_tokens=50)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    responses.append(response)
    print(f"Response: {response}")

df_response = pd.DataFrame(responses, columns=['response'])
df_response.to_csv("test_neuralnovel_qlora_responses.csv", index=False)

from google.colab import files
files.download("test_neuralnovel_qlora_responses.csv")

## Evaluation Metrics

In [ ]:
test_df_preds = pd.read_csv('test_neuralnovel_qlora_responses.csv')
test_df_trues = pd.read_csv('test_dataset.csv')

print(test_df_trues.head())
print(test_df_preds.head())

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd

# Convert true and predicted positions to arrays for scoring
y_true = test_df_trues["position"]
y_pred = test_df_preds["position"]

# Classification Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_true, y_pred, average='weighted', zero_division=1)

# Regression Metrics
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, y_pred)
r_squared = r2_score(y_true, y_pred)

# Print all metrics
print(f'Classification Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}')
print(f'Regression Metrics:\nMSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nR-squared: {r_squared}')